# ![argopy logo](https://avatars1.githubusercontent.com/t/3711886?s=90&v=4) Argo data python library

![build](https://github.com/euroargodev/argopy/workflows/build/badge.svg?branch=feature-fetch-local-ftp)
[![codecov](https://codecov.io/gh/euroargodev/argopy/branch/feature-fetch-local-ftp/graph/badge.svg)](https://codecov.io/gh/euroargodev/argopy)
[![Requirements Status](https://requires.io/github/euroargodev/argopy/requirements.svg?branch=feature-fetch-local-ftp)](https://requires.io/github/euroargodev/argopy/requirements/?branch=feature-fetch-local-ftp)

``argopy`` is a python library that aims to ease Argo data access, visualisation and manipulation for regular users as well as Argo experts and operators.

Several python packages exist: we are currently in the process of analysing how to merge these libraries toward a single powerfull tool. [List your tool here !](https://github.com/euroargodev/argopy/issues/3)

## Install

Since this is a library in active development, use direct install from this repo to benefit from the last version:

```bash
pip install git+http://github.com/euroargodev/argopy.git@master
```

The ``argopy`` library should work under all OS (Linux, Mac and Windows) and with python versions 3.6, 3.7 and 3.8.

**The libray is already installed to run this notebook**.

# Import

In [ ]:
import os
import numpy as np
import xarray as xr

import argopy
from argopy import DataFetcher as ArgoDataFetcher

## Fetching Argo Data
Note that the primary data model used to manipulate Argo data is [xarray](https://github.com/pydata/xarray).

Init the default data fetcher like:


In [ ]:
argo_loader = ArgoDataFetcher()
argo_loader

#### Space/time domain

In [ ]:
ds = argo_loader.region([-85,-45,10.,20.,0,10.]).to_xarray()
ds

you can also specify a date range:

In [ ]:
ds = argo_loader.region([-85,-45,10.,20.,0,1000.,'2012-01','2014-12']).to_xarray()
ds

#### Profiles of a given float

This fetch data for the float ID 6902746 and cycle number 34:

In [ ]:
ds = argo_loader.profile(6902746, 34).to_xarray()
ds

You can also retrieve a list of cycles:

In [ ]:
ds = argo_loader.profile(6902746, np.arange(12,45)).to_xarray()
ds

#### One or more floats

In [ ]:
ds = argo_loader.float(6902746).to_xarray()
ds

In [ ]:
ds = argo_loader.float([6902746, 6902747, 6902757, 6902766]).to_xarray()
ds

## More advanced data fetching

Two Argo data fetchers are available.

#### The Ifremer erddap
This is the recommended and default fetching backend.

Not that it requires an internet connection.

In [ ]:
argo_loader = ArgoDataFetcher(backend='erddap')
ds = argo_loader.profile(6902746, 34).to_xarray()
ds

#### A local copy of the GDAC ftp

**If you don't know what GDAC means, you should not go down this line...**

This is a more complex fetching backend since it requires you to have a local copy of files from the GDAC ftp server.

Note that Argo provides monthly snapshots of the entire datasets (eg: [2020/02](https://www.seanoe.org/data/00311/42182/#70590) and [full list here](http://www.argodatamgt.org/Access-to-data/Argo-DOI-Digital-Object-Identifier)). This is great for reproducibility.

You can also managed a [synchronised copy of the GDAC using rsync](http://www.argodatamgt.org/Access-to-data/Argo-GDAC-synchronization-service).

This data fetching method is usefull for Argo experts and operators, but also because it provides an offline access to the dataset.

To initiate the fetcher, you need to use the ``backend`` option and more importantly to specify the path toward the local copy of the ftp dataset (more specifically the path where all DAC are listed).

**For demonstration purposes**, we will rely here on a dummy sample of GDAC ftp files provides by the ``argopy`` tutorial module:

In [ ]:
ftproot, flist = argopy.tutorial.open_dataset('localftp')
print(ftproot)

Under this path, we have downloaded some Argo netcdf files. For instance:

In [ ]:
flist[14]

In [ ]:
argo_loader = ArgoDataFetcher(backend='localftp', path_ftp=os.path.join(ftproot, 'dac'))
ds = argo_loader.float(3902131).to_xarray()
ds

Note that to avoid specifying on each request the local ftp path copy, you can use the handy options setter:

In [ ]:
argopy.set_options(local_ftp=os.path.join(ftproot, 'dac'))

Options are valid only throughout a python session, so you have to set this option at the beginning of your script.

So now, we don't have to specify the path anymore:

In [ ]:
ArgoDataFetcher(backend='localftp').float(3902131).to_xarray()

## Fetched data manipulation

#### Point vs profile
As you probably have noticed now, data are returned as a collection of measurements along the dimension ``index``.

If you want to convert them into a collection of profiles, you can use methods from the xarray accessor named ``argo``:

In [ ]:
ds_as_points = ArgoDataFetcher().float(5903248).to_xarray()
ds_as_points

Now to convert this dataset into a collection of profiles:

In [ ]:
ds_as_profiles = ds_as_points.argo.point2profile()
ds_as_profiles

And finaly you can reverse that operation with the companion method:

In [ ]:
ds_as_profiles.argo.profile2point()

#### Other formats

By default fetched data are returned in memory as [xarray.DataSet](http://xarray.pydata.org/en/stable/data-structures.html#dataset). 
From there, it is easy to convert them to other formats like a [Pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe):

In [ ]:
ds = ArgoDataFetcher().profile(6902746, 34).to_xarray()
df = ds.to_dataframe()
df

## Export data to files

In [ ]:
ds = ArgoDataFetcher().region([-85,-45,10.,20.,0,100.,'2012-01','2012-12']).to_xarray()
ds

In [ ]:
ds.to_netcdf('my_selection.nc')
!ls -l my_selection.nc